# scRNA-seq Data

In this blog I'll be referencing a lot of products.  I'm most familiar with <strong style="color:#EB1960">Illumina</strong> but I'll try to give an overview of others too.  This will likely be biased towards <strong style="color:#EB1960">Illumina</strong>.

I'm heavily indebted to Morgan for help in understanding this topic.

## Library Preparation

<strong style="color:#537FBF">Library preparation</strong> is the collection of the bits of data that you're actually interested in (these bits would be your 'library').  For scRNA-seq we are interested in the <strong style="color:#A6A440">transcriptome</strong> (this is the set of RNA thingamajigs in the cell).  <strong style="color:#EB1960">@illumina-ebook</strong> lists a few library preparation techniques for the transcriptome:

<hr/>

*  Full-length RNA-seq
*  mRNA end-tag amplification
*  Targeted panels
   *  Targeted towards measuring specific things
*  IR-seq
   *  Specifically used for B and T cells.

<hr/>

These are important for me to understand so I'll definitely be delving into this topic deeper in the future.  The end result should be that the mRNA has been converted into cDNA.

## Sequencing

We've now isolated our cells and grabbed the bits that we're interested in (in cDNA format).  Now, we want to actually be able to learn about the mRNA that we've grabbed.  To do this, we need to sequence it. 

### Flow Cells

One important concept for sequencing is the <strong style="color:#A6A440">Flow Cell</strong>.  Intuitively, the flow cell is a sticky plate that will grab the mRNA.  <strong style="color:#EB1960">@illumina-ngs</strong> defines them as the following:

> <strong style="color:#A6A440">Flow Cell</strong>: A glass slide with one, two, or eight physically separated lanes, depending on the instrument platform. Each lane is coated with a lawn of surface bound, adapter-complimentary <strong style="color:#A6A440">oligos</strong>. A single library or a pool of up to 96 <strong style="color:#537FBF">multiplexed</strong> libraries
can be run per lane, depending on application parameters.
>
> -- <cite><strong style="color:#EB1960">Illumina handbook on next generation sequencing</strong></cite> [@illumina-ngs]



An <strong style="color:#A6A440">oligo</strong> is a short strand of synthetic DNA [@oligo].  These are the "sticky bits".  <strong style="color:#537FBF">Multiplexing</strong> is the process of sequencing multiple samples at a time; this can be useful as sequencing produces a lot of data, more than may be necessary for a single project on its own [@ngs-considerations].

![Example of a patterned flow cell](flow_cell.jpeg)

The above diagram is that of a 2D slice of a <strong style="color:#A6A440">Patterned Flow Cell</strong>.  These differ from <strong style="color:#A6A440">Nonpatterned Flow Cells</strong> in the use of <strong style="color:#A6A440">nanowells</strong> (little valleys in the cell) to ideally keep fragments of DNA from binding to nearby oligos.  Patterned flow cells are a more recent innovation, and for a brief overview of the differences you can read @patterned-vs-nonpatterned and @pvn-cons.

The cDNA created during library preparation is added onto the flow cell.  The strands of cDNA (<strong style="color:#A6A440">reads</strong>) will bind to the oligos, and then start multiplyling so that duplicates of the read will be bound to nearby oligos.  For patterned flow cells, this should mean that the nanowells are full of copies of the same read - for unpatterned flow cells the clusters are more chaotic.

Flow cells have two key measures (besides data quality); the amount of reads and the <strong style="color:#A6A440">read length</strong>.  Read counts can be in the hundreds of millions.  Read lengths will be much smaller; the flow cells paired with the <strong style="color:#EB1960">Illumina NextSeq 550</strong> can only have reads as long as 150 base pairs on average.  Longer reads are desirable from the perspective of reconstructing larger sequences, as it is easier to determine if two segments have a significant overlap.

An important concept is <strong style="color:#A6A440">coverage</strong>, or the amount of times a gene has a read associated with it.  Higher coverage is better as it allows us to piece together the gene more accurately and weed out incorrect bases.  This is also called <strong style="color:#A6A440">sequence depth</strong>.  Deep sequencing is important for detecting rare genes.  This can be affected by the steps done during library preparation; immune cells, which undergo <strong style="color:#537FBF">VDJ recombination</strong> - in short, each cell has its own unique markers that need to be accounted for [@ngs-considerations]

Finally, there's the decision of whether to use <strong style="color:#A6A440">paired-end</strong> or <strong style="color:#A6A440">single-end</strong> sequencing.  DNA and RNA are linear structures with two different ends - the 5' and 3' ends ("5-prime" and "3-prime").  By reading from both the ends, we can better find errors as well as increase the read length (intuitively: instead of reading n base pairs in one direction, you can read 2n total pairs with n in each direction).  These advantages decrease when you have <strong style="color:#A6A440">UMIs (unique molecular identifiers)</strong> or other feature you can take advantage of (such as VDJs), but that is out of scope for this blog post.  [@ngs-considerations]

### Sequencing Devices

#### Illumina NextSeq 550

### References

::: {#refs}
:::

<script src="https://giscus.app/client.js"
        data-repo="baileyandrew/blog"
        data-repo-id="R_kgDOInJwKg"
        data-category="Announcements"
        data-category-id="DIC_kwDOInJwKs4CTGOQ"
        data-mapping="title"
        data-strict="0"
        data-reactions-enabled="1"
        data-emit-metadata="0"
        data-input-position="top"
        data-theme="dark_protanopia"
        data-lang="en"
        data-loading="lazy"
        crossorigin="anonymous"
        async>
</script>